# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [1]:
import pandas as pd

---
## 2. Load data

In [2]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [3]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [4]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [5]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as yo can!

# Create the feature consumption trend

#### Why creating this feature is important
-> The consumption trends help in identifying changes in customer behaviour, which can signal potential churn risks.

1. When we have Declining Trend → then we have a high probability for Churn Risk
   
If a customer’s recent consumption is significantly lower than their yearly average (cons_last_month / cons_12m ≪ 1), they will be reducing their reliance on the company's services.

As a result the clients will surerly will be looking out switching providers, using alternative energy sources, or dissatisfaction with the service.

2. When the trend is stable then we have low risk and low probability for Churn (Stable Trend → Low Churn Risk)


If the ratio is close to 1, it means the customer is consuming at a consistent rate, suggesting a stable relationship with the company.

3. When the trend is increasing then we have A Potential opportunity to sell or upsell more services or products.(Increasing Trend → Potential Upselling Opportunity)

If the ratio we have calculated is bigger than 1, it could potentially indicate increased energy usage so the customer is dependent on us thus a more personalized and custom plans are a way to sell more services, understand their needs better and lower the risk for high margin customers


In [6]:
df["consumption_trend"] = df["cons_last_month"] / df["cons_12m"]

In [7]:
# Handle potential division by zero cases
df["consumption_trend"] = df["consumption_trend"].replace([float("inf"), -float("inf")], 0)
df["consumption_trend"] = df["consumption_trend"].fillna(0)

In [8]:
print(df[["cons_last_month", "cons_12m", "consumption_trend"]].head(5))

   cons_last_month  cons_12m  consumption_trend
0                0         0            0.00000
1                0      4660            0.00000
2                0       544            0.00000
3                0      1584            0.00000
4              526      4425            0.11887


# How to intepret the resuts 

#### Customers with consumption_trend = 0 → High risk of churn (they completely stopped using energy).
#### Customers with very low consumption_trend (<0.2) → Might be reducing usage, signaling dissatisfaction or switching providers.
#### Stable or increasing consumption_trend → Less likely to churn, as they are actively using the service.

In the first line we have no metrics since its customer 0 for just the index

The customer 1 has a yearly consumption but at the last month we had no consumption which indicates a potential churn and also it indicates that the client is looking for exit options or alternative provides.

The trend is the same for the 2 and 3 customer.

For the 4 customer he has a yearly consumption and has a last month consumption too around 12% of their typical year, so they are still consuming. But with 12% it can indicate a reduction in usage thus a potential churn or a customer not satisfied with pricing.

# Creating a feature to determine Actual vs Forecasted Usage
#### This feature help us understand how much energy a customer actually consumed compared to the forecasted consumption. It is usefull because it will detect unusual behaviour which will lead to assumptions and potential indicators about churning rates and probabilities 

### Why This Feature is Important for Churn Prediction

##### actual_vs_forecast < 1 (Low Usage)

The customer consumed less than expected, which may indicate dissatisfaction, switching providers, or economic issues.If this trend continues, they may completely stop consuming energy which will have as an impact the high churn risk

##### actual_vs_forecast = 0 (No Usage Despite Forecast)

This is a strong churn signal—the company expected the customer to use energy, but they did not used any thus they are opting out of our service and looking for potential new providers

##### actual_vs_forecast > 1 (High Usage)

If a customer consumes far more than forecasted, they will not opt out that easily and have less potential to leave our client.

In [9]:
# Create feature
df["actual_vs_forecast"] = df["cons_12m"] / df["forecast_cons_12m"]

In [10]:
# Handle potential division by zero cases
df["actual_vs_forecast"] = df["actual_vs_forecast"].replace([float("inf"), -float("inf")], 0)
df["actual_vs_forecast"] = df["actual_vs_forecast"].fillna(0)

In [11]:
print(df[["cons_12m", "forecast_cons_12m", "actual_vs_forecast"]].head())

   cons_12m  forecast_cons_12m  actual_vs_forecast
0         0               0.00            0.000000
1      4660             189.95           24.532772
2       544              47.96           11.342786
3      1584             240.04            6.598900
4      4425             445.75            9.927089


#### Explanation of results

The actual_vs_forecasted show how many times more hte consumption 12 months is higher than the forecasted

##### Customer 1
The forecasted consumption is 24 times less than the actual. The forecast here is not reliable

##### Customer 2
Higher consumption than forecasted by 11 times this signals that the customer is growing in usage.

##### Customer 3
The customer here consumed 6 times more than the forecast 

After inspecting the results it is safely to assume that the forecast method used by PowerCo company is not reliable, the model may be outdated or missing data.


# Average Annual Consuption per year of Subscription.

#### Here we measure a customers average yearly energy consumption over their loyal years to a subscriber. This will help PowerCo understand long-term customer behaviour and changes in behaviour.

## Why it is useful:
1. Helps identify long-term vs. short-term customers and their consumption habits.
2. Detects changes in energy usage over time (e.g., decreasing usage might indicate an upcoming churn).
3. Useful for customer segmentation based on loyalty and energy needs.

## Important For churning predictions

##### Low avg_annual_consumption → High Churn Risk

Customers who have been subscribed for many years but use very little energy may be:

1. Considering a switch to another provider.
2. Using alternative energy sources (e.g., solar panels).
3. No longer relying on the service.

##### High avg_annual_consumption → Low Churn Risk
1. Customers with high annual energy consumption are more dependent on the service and less likely to leave.
2. These are valuable, high-revenue customers who should be retained.

##### Sudden Drops in avg_annual_consumption → Warning Sign

If a long-term customer suddenly reduces their energy consumption, it might indicate:

1. A change in household/business needs
2. Dissatisfaction with the service
3. A plan to switch providers

In [12]:
# Compute avg_annual_consumption
df["avg_annual_consumption"] = df["cons_12m"] / df["num_years_antig"]

# Correct way to handle infinity and NaN values
df["avg_annual_consumption"] = df["avg_annual_consumption"].replace([float("inf"), -float("inf")], 0)
df["avg_annual_consumption"] = df["avg_annual_consumption"].fillna(0)

In [13]:
df[['id', 'channel_sales', 'cons_12m', 'num_years_antig', 'avg_annual_consumption']].head(5)

,id,channel_sales,cons_12m,num_years_antig,avg_annual_consumption
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,3,0.000000
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,6,776.666667
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,6,90.666667
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,6,264.000000
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,6,737.500000


# Intepretation of results

##### Customer 0
The customer is not using the service, no consumption for 12 months. This is a red flag which indicates that the customer has opted for another provider

##### Customer 1, Customer 3 and Customer 4
All these customers have moderate annual consumption, They are consuming and also they are long term users, if the consumption remains steady they will have less potential to churn

#### High consumption & longevity often correlates with customer retention. These customers may not be at high risk of churn unless they show a sudden drop in consumption.

#### Zero or low consumption clients may be at risk of churning. PowerCo should prioritize re-engagement or account review for these customers.

#### Investigating the lack of consumption could reveal issues like price dissatisfaction, service failure, or changes in customer behavior.


# Percentage Of Gas Consumption

##### It is important to understand PowerCo clients customers on their behaviour, how much overall energy their are using and how much is dependent on gas. This metric can provide valuable insights into customer behavior that could influence their churn rate, especially when gas and electricity have different pricing structures or service quality.

### Why the feature is important

##### Gas Dependence
If a customer relies more heavily on gas (the percentage of gas consumption is high), they may be more sensitive to issues related to gas supply, pricing changes, or service disruptions.

For example, if gas prices rise significantly or if there's an issue with gas availability, gas-dependent customers are more likely to churn in search of better pricing or more reliable service.

##### Electricity vs. Gas Consumption:
Customers with a balanced mix of electricity and gas usage may have more options to switch between different types of energy suppliers (if applicable) or energy solutions, which could affect their churn behavior.

For customers who rely mainly on one type of energy (gas), their loyalty to PowerCo could be more tied to the reliability and pricing of that specific service. If their gas consumption forms a large part of their total consumption and they face dissatisfaction with gas prices or service, they may consider switching providers.

##### Price Sensitivity:
This feature also acts as a proxy for price sensitivity. If a customer is highly dependent on gas, they may be more affected by price hikes or fluctuations in energy tariffs. Conversely, customers with minimal gas usage may be more focused on their electricity consumption and less sensitive to gas-related price changes.

In [14]:
df['gas_consumption_percentage'] = df.apply(
    lambda row: (row['cons_gas_12m'] / row['cons_12m']) * 100 if row['cons_12m'] != 0 else 0, axis=1
)

In [15]:
df[['cons_gas_12m', 'cons_12m', 'gas_consumption_percentage']].head(100)

,cons_gas_12m,cons_12m,gas_consumption_percentage
0,54946,0,0.000000
1,0,4660,0.000000
2,0,544,0.000000
3,0,1584,0.000000
4,0,4425,0.000000
...,...,...,...
95,0,15295,0.000000
96,0,18262,0.000000
97,0,7921,0.000000
98,24114,52988,45.508417


# Insights
It is good for targeting the small segment of our customers that are dependent on gas.

# Total Contact Duration Feature
##### The result is the total duration of the contract, which is in days, months, years, uncovering long term and short term customer. Newly acquired and loyal .

## Why it is important
Is important for understanding churn rate because it gives insights into customer loyalty, satisfaction, and the likelihood of churn based on how long a customer has been with the company.

##### Longer Contracts Often Indicate Higher Loyalty
Customers who have been with the company for a longer period (with a longer contract duration) tend to be more loyal. These customers are more likely to stay with the company, especially if their needs have been met over time.

Churn Insights: A shorter contract duration may indicate that the customer is newer, and possibly less committed to the service. Customers with short contract durations or who are nearing the end of their contract may be at a higher risk of churning when the contract expires.

##### Risk of Churn Increases Near Contract Expiry:
Interpretation: As customers approach the end of their contract, they may be evaluating whether to renew, switch providers, or change their services. The likelihood of churn often increases in this period if there are negative experiences or lack of renewal incentives.

Churn Insights: A shorter contract duration may suggest that a customer could be nearing contract renewal time, which is a critical period to retain them. Monitoring contract renewal dates and the duration before renewal can help the company act proactively, for example, by offering renewal incentives.

##### Customer Lifecycle Management
New customers (shorter contract duration) may need additional support to enhance satisfaction and avoid early churn.

Loyal customers (longer contract duration) may benefit from loyalty programs or discounts to keep them from switching providers.

Churn Insights: Customers who have been with the company for a longer period might be less sensitive to price changes or minor service disruptions, but may need different retention strategies to keep them satisfied over time.

##### Early Warning for Short-Term Contracts:
Interpretation: Short contracts (e.g., 6 months or 1 year) could imply that the customer may be testing the service or could be more likely to switch providers when their contract expires.

Churn Insights: Customers with short contracts may need additional incentives (e.g., loyalty programs, better pricing, customized offers) to encourage them to renew. Identifying early if the customer is in their contract's final months allows the company to take preventative measures before they churn.

In [16]:
# Convert to datetime the days in order to be able to work calculations
df['date_activ'] = pd.to_datetime(df['date_activ'])
df['date_end'] = pd.to_datetime(df['date_end'])

In [17]:
# Create the contract duration in days
df['contract_duration_days'] = (df['date_end'] - df['date_activ']).dt.days

In [18]:
# Contract duration in months
df['contract_duration_months'] = (df['date_end'].dt.year - df['date_activ'].dt.year) * 12 + (df['date_end'].dt.month - df['date_activ'].dt.month)

In [19]:
# Contract duration in years
df['contract_duration_years'] = df['contract_duration_months'] / 12

In [20]:
df[['id', 'date_activ', 'date_end', 'contract_duration_days', 'contract_duration_months', 'contract_duration_years']].head(5)

,id,date_activ,date_end,contract_duration_days,contract_duration_months,contract_duration_years
0,24011ae4ebbe3035111d65fa7c15bc57,2013-06-15,2016-06-15,1096,36,3.000000
1,d29c2c54acc38ff3c0614d0a653813dd,2009-08-21,2016-08-30,2566,84,7.000000
2,764c75f661154dac3a6c254cd082ea7d,2010-04-16,2016-04-16,2192,72,6.000000
3,bba03439a292a1e166f80264c16191cb,2010-03-30,2016-03-30,2192,72,6.000000
4,149d57cf92fc41cf94415803a877cb4b,2010-01-13,2016-03-07,2245,74,6.166667


### Summary of insights
##### Customers with contracts shorter than 1 year may have a higher churn rate because they’re more likely to explore alternatives.
##### Customers with contracts over 5 years might show lower churn rates due to established loyalty, but any churn in this group could be a critical loss.

# Price Sensitivity Variance Analysis
##### var_6m_price_peak_var: This represents the variance in the price of energy during peak hours over the last 6 months. Variance here measures how much the price has fluctuated during this period.
##### var_year_price_peak_var: This represents the variance in the price of energy during peak hours over the last year.

## Feature Explanation:
Calculates the ratio between the 6-month price variance and the 1-year price variance. A higher ratio suggests that prices are more volatile in the short term (6 months) compared to the long term (1 year), which could imply higher price sensitivity from customers in the short term.

## Result Explanation
##### High Ratio:
A higher value for this feature indicates that the price volatility in the last 6 months is higher compared to the price volatility over the past year. This may imply that customers are more sensitive to recent price fluctuations, which could increase the risk of churn, especially if recent price increases have been significant.

##### Low Ratio:
A lower value means that the 6-month price volatility is relatively stable when compared to the 1-year volatility, which may indicate that price sensitivity is lower, and customers may not be as affected by short-term fluctuations

## Why it is important for the Churn rate

##### Customer Price Sensitivity:
Customers who experience high price volatility in the short term (6 months) may feel uncertain about their future bills, which increases the likelihood of churn. This feature can help identify those at greater risk based on recent price fluctuations.

##### Price Stability:
If a customer is exposed to less price volatility in the last 6 months compared to the last year, they may feel more confident in sticking with the provider, reducing churn risk.

##### Churn Prediction:
Understanding the ratio of short-term to long-term price volatility can help predict which customers are more likely to churn based on their exposure to recent price changes.

In [21]:
df['price_sensitivity'] = df['var_6m_price_peak_var'] / df['var_year_price_peak_var']

In [22]:
df[[ 'var_6m_price_peak_var', 'var_year_price_peak_var', 'price_sensitivity']].head(5)

,var_6m_price_peak_var,var_year_price_peak_var,price_sensitivity
0,4.100838e-05,2.627605e-05,1.560675
1,1.217891e-03,6.089453e-04,2.000000
2,9.450150e-08,2.558511e-07,0.369361
3,0.000000e+00,0.000000e+00,NaN
4,2.896760e-06,3.552481e-06,0.815419


# Insights
Higher price_sensitivity (> 1): Customers with values greater than 1 (like row 0 and row 1) are more sensitive to recent price fluctuations. These customers may be more likely to churn due to the volatility of energy prices in the short term.

Lower price_sensitivity (< 1): Customers with values less than 1 (like row 2 and row 4) are less sensitive to recent price changes compared to longer-term changes, which could indicate lower churn risk.